<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/%E9%B3%A5%E3%81%AE%E9%B3%B4%E3%81%8D%E5%A3%B0%E5%88%86%E6%9E%90%EF%BC%88Google_Colaboratory%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Raven Proは、鳥の鳴き声分析において世界的に標準的なソフトウェアであり、特に**高精度な計測**や**自動検出**機能に優れています。toriRが提供するアノテーション機能が「どのような鳴き声がいつ発生したか」のデータ収集・蓄積を目的としているのに対し、Raven Proは収集したデータに対して「その鳴き声の周波数・時間特性はどうか」といった**詳細な音響計測**や**自動検出アルゴリズムの開発**を可能にします。

これらのRaven Proの高度な分析機能を、Google ColaboratoryでPythonライブラリ（主に`librosa`と`numpy`）を使って実現する方法を追記します。

-----

## 5\. Raven Proの機能の再現：音響計測と検出

### A. 詳細な音響計測（Selection Tableのシミュレーション）

Raven Proの「Selection Table」では、スペクトログラム上で矩形選択した鳴き声に対し、持続時間や周波数範囲など70以上の計測値を取得できます。ここでは、基本的な計測値をPythonで取得する方法を示します。

例として、`y`（音声データ）の**中央50%の部分**を「鳴き声のセレクション」として計測します。

In [ ]:
import numpy as np

# 音声データの長さ（秒）
duration_sec = len(y) / sr

# 計測したい「鳴き声のセレクション」のサンプル範囲を定義
# 例: 録音全体の中央50%の部分を「鳴き声」と仮定
start_ratio = 0.25
end_ratio = 0.75

start_sample = int(len(y) * start_ratio)
end_sample = int(len(y) * end_ratio)

# セレクションの音声データ
y_selection = y[start_sample:end_sample]

# 1. 持続時間 (Duration)
duration_selection_sec = len(y_selection) / sr
print(f"持続時間 (Duration): {duration_selection_sec:.3f} 秒")

# 2. 基本周波数 (Fundamental Frequency / Pitch)
# ピッチ検出アルゴリズム（例: YIN）を使用して、鳴き声の主な周波数を推定します。
# fminとfmaxは分析対象の鳥の声の周波数範囲に合わせて設定してください。
f0, _, _ = librosa.pyin(y_selection, sr=sr, fmin=100, fmax=8000)

# f0の平均値（欠損値を除外）を計測
f0_mean = np.nanmean(f0)
print(f"基本周波数 平均 (Mean Freq): {f0_mean:.1f} Hz")

# 3. 周波数範囲 (Frequency Range / Bandwidth)
# スペクトログラムの計算（以前のステップの S_dB を使用）
# 鳴き声部分のパワースペクトログラムを抽出
S_selection = S[:, int(start_sample/HOP_LENGTH) : int(end_sample/HOP_LENGTH)]

# 選択範囲の最大パワーを持つ周波数インデックスを取得
freqs = librosa.fft_frequencies(sr=sr, n_fft=N_FFT)
mean_power_spectrum = np.mean(librosa.amplitude_to_db(S_selection, ref=np.max), axis=1)

# 例えば、上位90%のエネルギーを含む周波数帯域を見つける、など、より複雑なロジックを実装できます。
# ここでは簡易的に、エネルギーのピークを持つ周波数を取得
peak_freq_index = np.argmax(mean_power_spectrum)
peak_freq = freqs[peak_freq_index]
print(f"ピーク周波数: {peak_freq:.1f} Hz")

### B. 自動検出（Band-Limited Energy Detector: BLEDのシミュレーション）

Raven ProのBLEDは、特定の**周波数帯域**で**ノイズレベルを上回るエネルギー**を持つイベントを自動で検出します。これは、特定の鳥種（例：低周波で鳴くフクロウ、高周波で鳴くコウモリ）を探すのに役立ちます。

In [ ]:
# --- BLEDの簡易シミュレーション ---

# 1. 検出したい周波数帯域をHzで定義 (例: 1,000 Hz から 4,000 Hz の帯域)
FREQ_MIN = 1000
FREQ_MAX = 4000

# 2. 検出用の閾値を設定 (dB)
THRESHOLD_DB = -30.0

# 3. 指定周波数帯域のエネルギーを時系列で計算
# スペクトログラム S から、指定した周波数インデックスを抽出
S_power = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, fmin=FREQ_MIN, fmax=FREQ_MAX)

# 時間軸で合計し、パワー時系列 (帯域内エネルギー) を得る
band_energy = np.sum(S_power, axis=0)

# 4. デシベルスケールに変換し、閾値処理
band_energy_db = librosa.power_to_db(band_energy, ref=np.max)

# 5. 検出されたフレームを見つける
detections = band_energy_db > THRESHOLD_DB

# --- 結果の可視化 ---
times = librosa.times_like(band_energy)
plt.figure(figsize=(12, 4))
plt.plot(times, band_energy_db, label='Band Energy (dB)')
plt.axhline(THRESHOLD_DB, color='r', linestyle='--', label='Threshold')
plt.scatter(times[detections], band_energy_db[detections], color='g', label='Detection')
plt.title(f'{FREQ_MIN}-{FREQ_MAX} Hz 帯域のエネルギー検出 (BLEDシミュレーション)')
plt.xlabel('Time (s)')
plt.ylabel('Energy (dB)')
plt.legend()
plt.show()

# 検出結果の要約
detection_time_frames = times[detections]
print(f"\n検出されたイベントのフレーム数: {len(detection_time_frames)}")
if len(detection_time_frames) > 0:
    print(f"最初の検出時刻: {detection_time_frames[0]:.2f} 秒")

### C. 機械学習検出器（Machine Learning Detector）

Raven Proの最新版には、BirdNETなどの既存の機械学習モデルを統合する機能が追加されています。

Google Colabでは、このBirdNETモデルを独立したPythonライブラリとして直接利用できます。`BirdNET-Analyzer`というライブラリを使用すれば、Colab上で音声ファイルを入力するだけで、数千種の鳥種の鳴き声を自動的に検出し、種名を推定することが可能です。

In [ ]:
# ColabでBirdNETを利用する場合（BirdNET-Analyzerのインストールと実行）
# !pip install birdnetlib
# from birdnetlib import Recording
# from birdnetlib.analyzer import Analyzer

# analyzer = Analyzer()
# recording = Recording(analyzer, FILENAME)
# recording.analyze()
# print(recording.detections)

これらの手法を用いることで、toriRで収集した鳴き声データや、Raven Proで分析したいと考えていた高度な計測・検出タスクを、Google ColaboratoryのPython環境で実現できます。

<div class="md-recitation">
  Sources
  <ol>
  <li><a href="https://www.ravensoundsoftware.com/software/raven-pro/">https://www.ravensoundsoftware.com/software/raven-pro/</a></li>
  </ol>
</div>